In [1]:
import polars as pl

In [2]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_inflacion_adj_ipc.parquet...
- [1 files][861.8 MiB/861.8 MiB]                                                
Operation completed over 1 objects/861.8 MiB.                                    


In [3]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_inflacion_adj_ipc.parquet'

In [18]:
df = pl.read_parquet(dataset_path + dataset_file)

In [19]:
# corrijo el tipo de estas 2:

df = df.with_columns([
    pl.col("tmobile_app").cast(pl.Int64),
    pl.col("cmobile_app_trx").cast(pl.Int64)
])

In [20]:
# elimino columnas con data drifting
rotas = ['cmobile_app_trx','tmobile_app','active_quarter','mplazo_fijo_pesos','minversion2']
df = df.select(pl.exclude(rotas))

In [21]:
# lista de columnas originales para operar despues
originales = df.select(pl.exclude(["numero_de_cliente", "foto_mes", "clase_ternaria"])).columns

In [ ]:
# lag 1
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag1")
    for col in originales
])

# lag 2
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(2)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag2")
    for col in originales
])

# lag 6
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(2)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag6")
    for col in originales
])

In [ ]:
# delta 1
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag1")).alias(f"{col}_delta1")
    for col in originales
])

# delta 2
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag2")).alias(f"{col}_delta2")
    for col in originales
])

# delta 6
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag6")).alias(f"{col}_delta6")
    for col in originales
])

In [27]:
dataset_path

'/home/eanegrin/datasets/'

In [28]:
df.write_parquet(dataset_path + 'competencia_03_fe_v05.parquet')

In [29]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v05.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v05.parquet...
- [1 files][  9.5 GiB/  9.5 GiB]                                                
Operation completed over 1 objects/9.5 GiB.                                      


In [ ]:
# Dataset con solo periodo de test y de prediccion para que nos de la RAM para levantarlo localmente
df = df.filter(pl.col("foto_mes").is_in([202107,202109]))

df.write_parquet(dataset_path + 'competencia_03_fe_v05_predict.parquet')

In [ ]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v05_predict.parquet /home/eanegrin/buckets/b1/datasets/